### Original Pipelieline

In [1]:
%pip install -U llama_index llama_index.llms.ollama llama_index.embeddings.ollama \
    llama_index.vector_stores.chroma llama-index-callbacks-arize-phoenix \
    llama_index.embeddings.huggingface PyPDF2 transformers torch sentence-transformers ipywidgets llama-index-llms-dashscope

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.llms.ollama import Ollama 
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
# Settings.embed_model = OllamaEmbedding(model_name="jinaai/jina-embeddings-v2-base-es")
# Settings.embed_model = OllamaEmbedding(model_name="qllama/bge-small-en-v1.5:f16")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
# Settings.llm = Ollama(model="qwen2.5:7b-instruct")
# Settings.llm = Ollama(model="qwen3", thinking=False, request_timeout=360.0)

# Settings.llm = Ollama(model="deepseek-r1:8b", thinking=False, request_timeout=360.0)
Settings.llm = Ollama(model="qwen3:1.7b", thinking=False, request_timeout=360.0)


# from llama_index.llms.dashscope import DashScope
# import os
# load_dotenv()
# Settings.llm = DashScope(model="qwen-plus", api_key=os.getenv("DASHSCOPE_API_KEY"))

In [3]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.ingestion import IngestionPipeline, IngestionCache, DocstoreStrategy
from llama_index.core.node_parser import SentenceSplitter
# from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.prompts import PromptTemplate
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.schema import TransformComponent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import nest_asyncio
import pandas as pd
import chromadb
import asyncio
import os
from dotenv import load_dotenv
import glob
import PyPDF2
import json
import re
from typing import ClassVar
from llama_index.core.extractors import TitleExtractor, QuestionsAnsweredExtractor
import random

data_path = "./data/"
project_name = "neus_catala"
file_prefix = re.sub('[^A-Za-z0-9]', '_', f"{project_name}_{Settings.embed_model.model_name}")
default_path = f"{data_path}{file_prefix}"

chroma_file = default_path + '_chroma.db' 
ingest_cache_file = default_path + "_cache.json"


db = chromadb.PersistentClient(path=chroma_file)
chroma_collection = db.get_or_create_collection(name=project_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
ingest_cache = IngestionCache()
if os.path.isfile(ingest_cache_file):
    ingest_cache.from_persist_path(ingest_cache_file)


class TextCleaner(TransformComponent):

    rx_nl: ClassVar  = re.compile(r'(\n\s*\n)')
    rx_hyphen: ClassVar  = re.compile(r'(-\n)')
    rx_notdot: ClassVar  = re.compile(r'(?<!\.)\n')
    rx_num_leter: ClassVar  = re.compile(r'(\d)([a-zA-Z])')

    def __call__(self, nodes, **kwargs):
        # nodes = list(map(lambda node: TextCleaner()(node), nodes))
        for node in nodes:
            text = self.rx_nl.sub('\n', node.text)
            text = self.rx_hyphen.sub('', text)
            text = self.rx_notdot.sub(' ', text)
            text = self.rx_num_leter.sub(r'\1 \2', text)   
            node.set_content(text)
        return nodes


pipeline = IngestionPipeline(
    transformations=[
        TextCleaner(),
        SentenceSplitter(chunk_size=512),
        Settings.embed_model,
    ],
    vector_store=vector_store,
    cache=ingest_cache,
)



### Digest Documents

In [4]:
from llama_index.core import SimpleDirectoryReader

input_dir = data_path + "documents"
already_processed_files = list(set( m['file_path'] for m in chroma_collection.get(include=['metadatas'])['metadatas']))

try:
    documents = SimpleDirectoryReader(
        input_dir=input_dir, 
        exclude=already_processed_files
        ).load_data()

    nodes = await pipeline.arun(documents=documents, show_progress=True)
    ingest_cache.persist(ingest_cache_file)
    len(nodes)
except ValueError:
    print('No new Documents')

No new Documents


## Evaluation

### Generation of the pairs Question / answer

In [ ]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.llama_dataset.legacy.embedding import DEFAULT_QA_GENERATE_PROMPT_TMPL
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
import os
import torch
torch.cuda.empty_cache()
index = VectorStoreIndex.from_vector_store(vector_store)

qa_file = default_path + "_qa.json"
regenerate = False
if not os.path.isfile(qa_file) or regenerate:
    
    nodes = [item.node for item in index.as_retriever(similarity_top_k=1000000).retrieve("")]
    # nodes = [node for node in nodes if node.metadata.get("file_name") == "NeusCatala-llibre.pdf"]
    nodes = [node for node in nodes if len(node.text) > 255]
    
    qa_dataset = generate_question_context_pairs(
        nodes=random.sample(nodes, min(len(nodes), 50)), 
        num_questions_per_chunk=1,
        qa_generate_prompt_tmpl=DEFAULT_QA_GENERATE_PROMPT_TMPL+ "Use Catalan language. do not add any introductory text or formatting. /no_think",
    )
    qa_dataset.save_json(qa_file)


100%|██████████| 50/50 [00:51<00:00,  1.03s/it]


### Generation of the responses

In [25]:
from llama_index.core.evaluation import RetrieverEvaluator, EmbeddingQAFinetuneDataset

# Convert to retriever
retriever = index.as_retriever(similarity_top_k=5)
# Define metrics
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

# Create RetrieverEvaluator
retriever_evaluator = RetrieverEvaluator.from_metric_names(metric_names=metrics, retriever=retriever)
qa_dataset = EmbeddingQAFinetuneDataset.from_json(qa_file)

eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset, show_progress=True)



100%|██████████| 50/50 [00:00<00:00, 98.45it/s] 


In [27]:

import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

display_results("top-2 eval", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-2 eval,0.34,0.242333,0.068,0.34,0.242333,0.266196


In [ ]:
# import nest_asyncio
from IPython.display import display, HTML
import nest_asyncio

queries = random.sample(list(qa_dataset.queries.items()),1)
# queries = [("202f20ac-f9d6-4278-a25e-41311d437c33",qa_dataset.queries["202f20ac-f9d6-4278-a25e-41311d437c33"])]
for (sample_id, sample_query) in queries:
    sample_expected = qa_dataset.relevant_docs[sample_id]

    nest_asyncio.apply()
    eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
    display(HTML(f'{eval_result}'))

    print('📄 Expected Nodes:')
    for id in eval_result.expected_ids:
        for node in index.vector_store.get_nodes(id):
            display(HTML(f"{id}: {node.text} ({node.metadata.get('file_name', 'N/A')})"))

    print('eval_result',eval_result) 
    
    print('📄 Retrieved Nodes:')
    for id in eval_result.retrieved_ids:
        for node in index.vector_store.get_nodes(id):
            display(HTML(f"{id}: {node.text} ({node.metadata.get('file_name', 'N/A')})"))

    nest_asyncio.apply()
    query_engine = index.as_query_engine()
    response = query_engine.query(sample_query)
    display(HTML(f'{response}'))


📄 Expected Nodes:


eval_result Query: Quina importància tenia que l'acompanyant seleccionat reunís les condicions de major confiança política i la qualificació mèdica, segons el text?
Metrics: {'hit_rate': 0.0, 'mrr': 0.0, 'precision': 0.0, 'recall': 0.0, 'ap': 0.0, 'ndcg': 0.0}

📄 Retrieved Nodes:
